In [1]:
import dask.dataframe as dd
#import cudf
from dask.distributed import LocalCluster, Client
#from dask_cuda import LocalCUDACluster
#import dask_cudf 

import glob

DATA_PATH = "./data/"

features = [
    'text_tokens',    ###############
    'hashtags',       #Tweet Features
    'tweet_id',       #
    'media',          #
    'links',          #
    'domains',        #
    'tweet_type',     #
    'language',       #
    'timestamp',      ###############
    'a_user_id',              ###########################
    'a_follower_count',       #Engaged With User Features
    'a_following_count',      #
    'a_is_verified',          #
    'a_account_creation',     ###########################
    'b_user_id',              #######################
    'b_follower_count',       #Engaging User Features
    'b_following_count',      #
    'b_is_verified',          #
    'b_account_creation',     #######################
    'b_follows_a',    #################### Engagement Features
    'reply',          #Target Reply
    'retweet',        #Target Retweet    
    'retweet_comment',#Target Retweet with comment
    'like',           #Target Like
                      ####################
]



In [2]:
all_tsv_files = glob.glob(DATA_PATH+"*.tsv")

In [3]:
cluster = LocalCluster(ip='0.0.0.0', local_directory='/home/rostislav/dask')
client = Client(cluster)
client

Client Scheduler: tcp://192.168.0.102:38041 Dashboard: http://192.168.0.102:8787/status,Cluster Workers: 6 Cores: 24 Memory: 31.33 GiB


In [4]:
df = dd.read_csv(all_tsv_files, header=None, names=features, sep='\x01',usecols=['tweet_id', 'b_user_id', 'language'], )
df = client.persist(df)

TypeError: read_csv() got an unexpected keyword argument 'columns'

In [6]:
unique_tweets = df['tweet_id'].nunique().compute()

KilledWorker: ("('read-csv-3c753244a60c9ba3425440377604af60', 5464)", <Worker 'tcp://192.168.0.102:46171', name: 0, memory: 0, processing: 1510>)

In [ ]:
unique_users = df['b_user_id'].nunique().compute()

In [ ]:
print("#unique users: {}, #unique tweets:{}".format(unique_users, unique_tweets))

In [ ]:
def calculate_quantiles_percentile(dataframe, col_label):
    interactions = dataframe.groupby(col_label).size().compute()
    
    median = interactions.median()
    third_quantile = interactions.quantile(0.75)
    ninetyth_percentile = interactions.quantile(0.9)
    
    return median, third_quantile, ninetyth_percentile 

In [ ]:
user_int_med, user_int_3rd, user_int_90th = calculate_quantiles_percentile(df, 'b_user_id')

print("User interactions: median={}, 3rd quantile={}, 90th percentile={}".format(user_int_med, user_int_3rd, user_int_90th))

In [ ]:
item_int_med, item_int_3rd, item_int_90th = calculate_quantiles_percentile(df, 'tweet_id')

print("User interactions: median={}, 3rd quantile={}, 90th percentile={}".format(item_int_med, item_int_3rd, item_int_90th))

In [ ]:
unique_languages = df['language'].nunique().compute()

In [ ]:
print("#languages={}".format(unique_languages))